In [10]:
import squigglepy as sq
from squigglepy.numbers import K, M, B
import numpy as np
import copy
import general_equations as ge

SIMULATIONS = 10*M


def dalys_per_1000_by_reducing_risk_mixture_model(models_dict, weights_dict, power):
    p_success, risk, bp_per_billion_if_success, times_credit_if_success = ge.create_variables(models_dict, weights_dict)
    dalys_risk_weighted = ge.one_model_dalys_per_1000_by_reducing_risk(risk, power, p_success, bp_per_billion_if_success, times_credit_if_success)

    return dalys_risk_weighted

def find_percentile_of_value(value, array_of_values):
    prop_under =  sum(array_of_values < value)/len(array_of_values)
    return prop_under

def print_outputs(dalys_per_1000_unweighted, risk_function, interpretation_function, const):
    dalys_per_1000_ghd = ge.dalys_per_1000_ghd_intervention()

    ev_ghd_dalys_per_1000 = np.mean(dalys_per_1000_ghd)
    ev_dalys_per_1000 = np.mean(dalys_per_1000_unweighted)
    print("Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: {}".format(ev_dalys_per_1000))
    print("Standard deviation DALYs/$1000 for no risk-aversion: {}".format(np.std(dalys_per_1000_unweighted)))

    print("Expected Value DALYs/$1000 if you choose GHD, No risk-aversion: {}".format(ev_ghd_dalys_per_1000))
    print("Standard deviation DALYs/$1000 for no risk-aversion: {}".format(np.std(dalys_per_1000_ghd)))

    rw_dalys_per_1000 = risk_function(dalys_per_1000_unweighted, const)
    rw_ghd_dalys_per_1000 = risk_function(dalys_per_1000_ghd, const)

    ev_rw_dalys_per_1000 = np.mean(rw_dalys_per_1000)
    ev_rw_ghd_dalys_per_1000 = np.mean(rw_ghd_dalys_per_1000)

    print("Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = {}: {}".format(const, ev_rw_dalys_per_1000))

    print("GHD Risk-Weighted DALYs/$1000, shape = {}: {}".format(const, ev_rw_ghd_dalys_per_1000))

    if ev_rw_dalys_per_1000 > ev_rw_ghd_dalys_per_1000: 
        print("X-risk reduction is better than GHD in terms of Risk-weighted expected value")
    elif ev_rw_dalys_per_1000 < ev_rw_ghd_dalys_per_1000:
        print("GHD is better than X-risk reduction in terms of Risk-weighted expected value")
    else:
        print("X-risk reduction and GHD are equally good in terms of Risk-weighted expected value")

    p_needed = interpretation_function(const)
    return ev_rw_dalys_per_1000, ev_rw_ghd_dalys_per_1000, p_needed

def get_percentiles_to_make_equal_to_ghd(dalys_per_1000_unweighted, risk_function, interpretation_function, c=0):
    dalys_per_1000_ghd = ge.dalys_per_1000_ghd_intervention()
    ev_ghd_dalys_per_1000 = np.mean(dalys_per_1000_ghd)
    print("Mean DALYs per $1000 if you choose GHD: ", ev_ghd_dalys_per_1000)
    prop_under = find_percentile_of_value(ev_ghd_dalys_per_1000, dalys_per_1000_unweighted)
    print("If you choose X-risk, there's a {} chance of saving fewer lives than the mean GHD spending".format(ge.get_pct(prop_under)))

    ev_rw_dalys_per_1000, ev_rw_ghd_dalys_per_1000, p_needed = print_outputs(dalys_per_1000_unweighted, risk_function, interpretation_function, c)
    return ev_rw_dalys_per_1000, ev_rw_ghd_dalys_per_1000, p_needed
    

## Models

In [11]:
## Population Times the Present We Can Take Credit For if we Solve X-risk
low_pop_credit = sq.lognorm(10, 1000)
high_pop_credit = sq.lognorm(47.5, 40000)
pop_weights = [0.9, 0.1]
pop_models = [low_pop_credit, high_pop_credit]

## Probability of Success
p_success = sq.lognorm(0.4,0.8, lclip=0.01, rclip=0.99)
p_success_models = [p_success]
p_success_weights = [1]

## Risk
risk = sq.lognorm(0.1, 0.25, lclip=0.01, rclip=0.5)
risk_models = [risk]
risk_weights = [1]

## Basis Points Per Billion If Success
bp_model1 = sq.lognorm(0.001, 0.001)
bp_model2= sq.lognorm(0.001, 2.2)
bp_model1n = sq.uniform(-0.0005, -0.00005)
bp_model2n= sq.norm(-.5, -0.0005)

bp_models = [bp_model1, bp_model2, bp_model1n, bp_model2n]
bp_per_billion_weights = [0.9/2, 0.1/2, 0.95/2, 0.05/2]

models = {'p_success': p_success_models, 'risk': risk_models, 
        'bp_per_billion_if_success': bp_models, 'times_credit_if_success': pop_models}

weights = {'p_success': p_success_weights, 'risk': risk_weights, 
           'bp_per_billion_if_success': bp_per_billion_weights, 'times_credit_if_success': pop_weights}


## CRRA Utility Function

#### Equations

In [12]:

def crra_interpretation(const):
    if const == 1:
        p_needed = np.log(100)/np.log(10*K)
    else:
        p_needed = (100**(1-const)-1**(1-const))/((10*K)**(1-const)-1**(1-const))

    return p_needed

def crra_risk_aversion(dalys, c=0):
    dalys2 = copy.deepcopy(dalys)
    if 0 < c < 1:
        for idx, value in enumerate(dalys):
            if value > 0:
                dalys2[idx] = (value**(1-c)-1)/(1-c)
            else:
                dalys2[idx] = -(15**(1-c)-1)/(1-c)
    elif c == 1: 
        for idx, value in enumerate(dalys):
            if value > 0:
                dalys2[idx] = np.log(value)
            else:
                dalys2[idx]  = -np.log(15)

    elif c == 0:
        dalys2 = dalys
    return np.array(dalys2)

def multiple_crra_risk_aversion(models, weights):
    coefficients = [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.35, 0.38, 0.4, 0.42, 0.43, 0.44, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    dalys_per_1000_unweighted = dalys_per_1000_by_reducing_risk_mixture_model(models, weights, 1)
    for c in coefficients:
        print("-------------------------------------------------------------------")
        if c == 0:
            print("Coefficient of Relative Risk Aversion: c = {} (No Risk Aversion)".format(c))
        else:
            print("Coefficient of Relative Risk Aversion: c = {}".format(c))
        ev_rw_dalys_per_1000, ev_rw_ghd_dalys_per_1000, p_needed = get_percentiles_to_make_equal_to_ghd(dalys_per_1000_unweighted, crra_risk_aversion, crra_interpretation, c)
        print("This level of risk aversion is like being indifferent to choosing to take $100 with certainty or a {} chance of getting $10K and {} chance of getting $1".format(ge.get_pct(p_needed), ge.get_pct(1-p_needed)))

#### CRRA Utility

In [13]:
multiple_crra_risk_aversion(models, weights)


100%|██████████| 1000000/1000000 [00:00<00:00, 1031355.46it/s]


-------------------------------------------------------------------
Coefficient of Relative Risk Aversion: c = 0 (No Risk Aversion)
Mean DALYs per $1000 if you choose GHD:  15.358260721017619
If you choose X-risk, there's a 90.41% chance of saving fewer lives than the mean GHD spending
Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: 1137.8993498737411
Standard deviation DALYs/$1000 for no risk-aversion: 242065.41091397157
Expected Value DALYs/$1000 if you choose GHD, No risk-aversion: 15.36754194318869
Standard deviation DALYs/$1000 for no risk-aversion: 4.686119286759498
Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = 0: 1137.8993498737411
GHD Risk-Weighted DALYs/$1000, shape = 0: 15.36754194318869
X-risk reduction is better than GHD in terms of Risk-weighted expected value
This level of risk aversion is like being indifferent to choosing to take $100 with certainty or a 0.99% chance of getting $10K and 99.01% chance of getting $1
------------------------

## Prospect Theory Method

#### Equations

In [14]:
def prospect_theory_utility(dalys, cs): 
    a, b, l = cs
    dalys2 = copy.deepcopy(dalys)
    for idx, value in enumerate(dalys):
        if value > 0:
            dalys2[idx] = (value)**a
        else:
            dalys2[idx] = -l*(-1*value)**b

    return np.array(dalys2)

def prospect_theory_utility_estimates(models, weights, coefficients, loss_aversion):
    c_tuples = [(a, a, l) for a in coefficients for l in loss_aversion]
    dalys_per_1000_unweighted = dalys_per_1000_by_reducing_risk_mixture_model(models, weights, 1)
    for cs in c_tuples:
        a, b, l = cs
        print("-------------------------------------------------------------------")
        print("Coefficients of Prospect Theory: a = {}, b = {}, l = {}".format(a, b, l))
        rw_ev_dalys_per_1000, rw_ghd_ev_dalys_per_1000, p_needed = get_percentiles_to_make_equal_to_ghd(dalys_per_1000_unweighted, prospect_theory_utility, prospect_theory_indifference, cs)
        print("This level of risk aversion and loss aversion is like being indifferent between choosing to take $100 with certainty versus {} chance of getting $10K versus nothing".format(ge.get_pct(p_needed)))

def prospect_theory_indifference(cs):
    a, b, l = cs
    p = (l*100**b)/(l*100**b+9900**a)
    return p 

coefficients = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.62, 0.63, 0.64, 0.65, 0.675, 0.7, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.8, 0.82, 0.85, 0.88, 0.9, 1]

#### Loss Aversion Not Allowed

In [15]:
loss_aversion=[1]
prospect_theory_utility_estimates(models, weights, coefficients, loss_aversion)

100%|██████████| 1000000/1000000 [00:01<00:00, 617068.94it/s]


-------------------------------------------------------------------
Coefficients of Prospect Theory: a = 0.01, b = 0.01, l = 1
Mean DALYs per $1000 if you choose GHD:  15.357080168770693
If you choose X-risk, there's a 90.34% chance of saving fewer lives than the mean GHD spending
Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: 1214.211230180214
Standard deviation DALYs/$1000 for no risk-aversion: 207118.22082646514
Expected Value DALYs/$1000 if you choose GHD, No risk-aversion: 15.366820033197524
Standard deviation DALYs/$1000 for no risk-aversion: 4.691188976520351
Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = (0.01, 0.01, 1): 0.006603776941050498
GHD Risk-Weighted DALYs/$1000, shape = (0.01, 0.01, 1): 1.02724588045385
GHD is better than X-risk reduction in terms of Risk-weighted expected value
This level of risk aversion and loss aversion is like being indifferent between choosing to take $100 with certainty versus 48.85% chance of getting $10K versus

#### Loss Aversion Allowed

In [16]:
loss_aversion=[2.25]
prospect_theory_utility_estimates(models, weights, coefficients, loss_aversion)

100%|██████████| 1000000/1000000 [00:01<00:00, 848317.68it/s]


-------------------------------------------------------------------
Coefficients of Prospect Theory: a = 0.01, b = 0.01, l = 2.25
Mean DALYs per $1000 if you choose GHD:  15.361502135708575
If you choose X-risk, there's a 90.34% chance of saving fewer lives than the mean GHD spending
Expected Value DALYs/$1000 if you choose X-risk, No risk-aversion: 1675.03302457975
Standard deviation DALYs/$1000 for no risk-aversion: 518152.6843792381
Expected Value DALYs/$1000 if you choose GHD, No risk-aversion: 15.365463995824502
Standard deviation DALYs/$1000 for no risk-aversion: 4.688324360631542
Risk-Weighted EV DALYs/$1000 if you choose X-risk, shape = (0.01, 0.01, 2.25): -0.35633741142330655
GHD Risk-Weighted DALYs/$1000, shape = (0.01, 0.01, 2.25): 1.027245165213413
GHD is better than X-risk reduction in terms of Risk-weighted expected value
This level of risk aversion and loss aversion is like being indifferent between choosing to take $100 with certainty versus 68.24% chance of getting $10

## Sensitivity Analysis

In [17]:
import sensitivity_analysis

sensitivity_analysis.lt_dalys_per_1000_sensitivity()

[1st Order] Sensitivity analysis for DALYs per $1000 spending on LT
LT DALYs per $1000 Sensitivity Analysis
probability_success : 0.021 +/- 0.022
bp_per_billion_if_success : 0.233 +/- 0.03
times_credit_if_success : 0.132 +/- 0.026
---------------------------------------------------------------------------
[Total] Sensitivity analysis for DALYs per $1000 spending on LT
LT DALYs per $1000 Sensitivity Analysis
probability_success : 0.524 +/- 0.03
bp_per_billion_if_success : 0.831 +/- 0.035
times_credit_if_success : 0.731 +/- 0.033
---------------------------------------------------------------------------


## Tests


In [18]:

import unittest
import os

class TestFunctions(unittest.TestCase):

    def test_value_gained_by_reducing_risk(self):
        pass_test = True

        p_success, risk, bp_per_billion_if_success, times_credit_if_success = ge.create_variables(models, weights)
        value_of_status_quo = ge.years_healthy_per_year()*ge.lifespan_person()*ge.world_population()*times_credit_if_success
        power = 1

        did_succeed = ge.generate_success_bernoullis(p_success)
        
        x_risk_reduction = did_succeed*bp_per_billion_if_success*ge.one_basis_point()*1*K/(1*B)

        value_gained = ge.value_gained_by_reducing_x_risk(risk, x_risk_reduction, value_of_status_quo, power)
        expected_value_gained = x_risk_reduction*value_of_status_quo
        
        if np.median(value_gained) < 0.9*np.median(expected_value_gained) or np.median(value_gained) > 1.1*np.median(expected_value_gained):
            pass_test = False
            print("x-risk reduced: {}, value status quo: {}".format(x_risk_reduction, value_of_status_quo))
            print("Expected value gained: ", expected_value_gained)
            print("Got value gained: ", value_gained)

        self.assertTrue(pass_test)

    def test_dalys_when_p_success_zero(self):
        pass_test = True
        p_success, risk, bp_per_billion_if_success, times_credit_if_success = ge.create_variables(models, weights)
        p_success = np.zeros(SIMULATIONS)

        dalys = ge.one_model_dalys_per_1000_by_reducing_risk(risk, 1, p_success, bp_per_billion_if_success, times_credit_if_success)

        if dalys.all() != 0:
            pass_test = False
            print("DALYs when p_success is zero: ", dalys)

        self.assertTrue(pass_test)

    def test_crra_close_when_equal(self):
        pass_test = True
        outside = 0
        count = 0
        outside_c = []
        for c in coefficients:
            dalys_per_1000_lt = ge.dalys_per_1000_ghd_intervention()

            dalys_per_1000_ghd = ge.dalys_per_1000_ghd_intervention()

            ev_ghd_dalys_per_1000 = np.mean(dalys_per_1000_ghd)
            ev_dalys_per_1000 = np.mean(dalys_per_1000_lt)

            rw_dalys_per_1000 = crra_risk_aversion(dalys_per_1000_lt, c)
            rw_ghd_dalys_per_1000 = crra_risk_aversion(dalys_per_1000_ghd, c)

            ev_rw_dalys_per_1000 = np.mean(rw_dalys_per_1000)
            ev_rw_ghd_dalys_per_1000 = np.mean(rw_ghd_dalys_per_1000)

            count += 2
            if ev_rw_dalys_per_1000 < 0.9*ev_rw_ghd_dalys_per_1000 or ev_rw_dalys_per_1000 > 1.1*ev_rw_ghd_dalys_per_1000:
                outside += 1
                outside_c.append(c)
            if ev_dalys_per_1000 < 0.9*ev_ghd_dalys_per_1000 or ev_dalys_per_1000 > 1.1*ev_ghd_dalys_per_1000:
                outside += 1
                outside_c.append(c)
        if outside/count > 0.2:
            pass_test = False
            print("Outside 20% of the time")
        
        self.assertTrue(pass_test)

    def test_prospect_theory_close_when_equal(self):
        pass_test = True
        outside = 0
        count = 0
        outside_c = []
        loss_aversion = [1, 2.25]
        for c in coefficients:
            for l in loss_aversion:
                cs = [c, c, l]
                dalys_per_1000_lt = ge.dalys_per_1000_ghd_intervention()

                dalys_per_1000_ghd = ge.dalys_per_1000_ghd_intervention()

                ev_ghd_dalys_per_1000 = np.mean(dalys_per_1000_ghd)
                ev_dalys_per_1000 = np.mean(dalys_per_1000_lt)

                rw_dalys_per_1000 = prospect_theory_utility(dalys_per_1000_lt, cs)
                rw_ghd_dalys_per_1000 = prospect_theory_utility(dalys_per_1000_ghd, cs)

                ev_rw_dalys_per_1000 = np.mean(rw_dalys_per_1000)
                ev_rw_ghd_dalys_per_1000 = np.mean(rw_ghd_dalys_per_1000)

                count += 2
                if ev_rw_dalys_per_1000 < 0.9*ev_rw_ghd_dalys_per_1000 or ev_rw_dalys_per_1000 > 1.1*ev_rw_ghd_dalys_per_1000:
                    outside += 1
                    outside_c.append(c)
                if ev_dalys_per_1000 < 0.9*ev_ghd_dalys_per_1000 or ev_dalys_per_1000 > 1.1*ev_ghd_dalys_per_1000:
                    outside += 1
                    outside_c.append(c)
        if outside/count > 0.2:
            pass_test = False
            print("Outside 20% of the time")
        
        self.assertTrue(pass_test)

    def test_prospect_interpretation(self):
        pass_test = False
        for l in [1, 2.25]:
            for a in coefficients:
                p_10k = l*100**a/(l*100**a+9900**a)
                u_10k = 9900**a
                u_0 = -1*l*100**a

                u_100 = p_10k*u_10k + (1-p_10k)*u_0

                if round(u_100,2) !=0:
                    pass_test = False
                    print("Back calculated probability wrong")
                    print(u_100)
                p = prospect_theory_indifference([a, a, l])

                u_100_2 = p*u_10k + (1-p)*u_0
                if round(u_100_2,2) != 0:
                    pass_test = False
                    print("Calculated probability wrong")
            
                if round(p,2) != round(p_10k,2):
                    pass_test = False
                    print("Probabilities not equal")

    def test_crra_interpretation(self):
        pass_test = False
        for c in coefficients:
            if c != 1:
                p_10k = ((100)**(1-c)-1**(1-c))/((10*K)**(1-c)-1**(1-c))
                u_10k = ((10*K)**(1-c)-1)/(1-c)
                u_1 = ((1)**(1-c)-1)/(1-c)

                u_100 = p_10k*u_10k + (1-p_10k)*u_1

                if round(u_100) != round((100**(1-c)-1)/(1-c)):
                    pass_test = False
                    print("Back calculated probability wrong")
                    print(c)
                    print(u_100)
                    print("Supposed to be: {}".format((100**(1-c)-1)/(1-c)))
                p = crra_interpretation(c)

                u_100_2 = p*u_10k + (1-p)*u_1
                if round(u_100_2) != round(u_100):
                    pass_test = False
                    print("Calculated probability wrong")
            
                if round(p,2) != round(p_10k,2):
                    pass_test = False
                    print("Probabilities not equal")
            else: 
                p_10k = np.log(100)/np.log(10*K)

                u_10k = np.log(10*K)
                u_1 = np.log(1)
                u_100 = p_10k*u_10k + (1-p_10k)*u_1

                if round(u_100) != round(np.log(100)):
                    pass_test = False
                    print("Back calculated utility wrong")
                    print(c)
                    print("Calculated: {}".format(u_100))
                    print("Supposed to be: {}".format(np.log(100)))
                p = crra_interpretation(c)

                u_100_2 = p*u_10k + (1-p)*u_1
                if round(u_100_2) != round(u_100):
                    pass_test = False
                    print("Calculated probability wrong")
            
                if round(p,2) != round(p_10k,2):
                    pass_test = False
                    print(c)
                    print("Probabilities not equal")
                    print(p)
                    print(p_10k)


res = unittest.main(argv=[''], verbosity=3, exit=False)

test_crra_close_when_equal (__main__.TestFunctions) ... ok
test_crra_interpretation (__main__.TestFunctions) ... ok
100%|██████████| 1000000/1000000 [00:00<00:00, 1271775.84it/s]
ok
test_prospect_interpretation (__main__.TestFunctions) ... ok
test_prospect_theory_close_when_equal (__main__.TestFunctions) ... ok
100%|██████████| 1000000/1000000 [00:00<00:00, 1264908.57it/s]
ok

----------------------------------------------------------------------
Ran 6 tests in 113.787s

OK
